In [1]:
import pandas as pd
import numpy as np
import os

# ============================================================
# 🧭 Y14_경찰청 치안센터 전처리 (법정동코드 기반)
# - 파일 로드
# - 법정동코드로 시·구·동 파생
# - 서울특별시 데이터만 필터링
# - 결측치 확인 및 샘플 출력
# - 지역구별, 동별 설치 수 집계
# - 최종 테이블 생성 및 저장
# ============================================================

# ===== 0) 경로 =====
input_file  = "/Users/mac/Documents/SORA_Project/data/raw/Y14_경찰청_전국 치안센터 주소 현황_20250630_TM.csv"
output_dir  = "/Users/mac/Documents/SORA_Project/data/preprocessing"
os.makedirs(output_dir, exist_ok=True)  # 폴더가 없으면 자동 생성

output_file = os.path.join(output_dir, "Y14_경찰청_전국 치안센터 주소 현황_20250630_TM_전처리.csv")

# ===== 1) 데이터 로드 =====
df = pd.read_csv(input_file, sep="\t", encoding="utf-8-sig")
print(f"✅ 원본 데이터 로드 완료: {df.shape}")
print("\n📌 데이터 기본 정보:")
df.info()


✅ 원본 데이터 로드 완료: (741, 18)

📌 데이터 기본 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 741 entries, 0 to 740
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   연번      741 non-null    int64  
 1   시도청     741 non-null    object 
 2   경찰서     741 non-null    object 
 3   치안센터명   741 non-null    object 
 4   주소      741 non-null    object 
 5   입력주소    741 non-null    object 
 6   X       741 non-null    int64  
 7   Y       741 non-null    int64  
 8   CLSS    741 non-null    object 
 9   PNU     711 non-null    float64
 10  주소구분    741 non-null    object 
 11  표준신주소   680 non-null    object 
 12  표준구주소   715 non-null    object 
 13  우편번호    666 non-null    float64
 14  행정동코드   688 non-null    float64
 15  행정동명    688 non-null    object 
 16  법정동코드   687 non-null    float64
 17  법정동명    687 non-null    object 
dtypes: float64(4), int64(3), object(11)
memory usage: 104.3+ KB


In [2]:
# ===== 2) 법정동코드로 시·구·동 파생 =====
df["법정동코드_10자리"] = (
    df["법정동코드"]
    .fillna(0)
    .astype(np.int64)
    .astype(str)
    .str.zfill(10)
)
df["시도코드"] = df["법정동코드_10자리"].str[:2]
df["시군구코드"] = df["법정동코드_10자리"].str[2:5]
df["법정동_코드"] = df["법정동코드_10자리"].str[5:10]

# 시 매핑
sido_map = {"11": "서울특별시"}  
df["시"] = df["시도코드"].map(sido_map)

# 구 매핑
gu_map = {
    '110': '종로구', '140': '중구', '170': '용산구', '200': '성동구', '215': '광진구',
    '230': '동대문구', '260': '중랑구', '290': '성북구', '305': '강북구', '320': '도봉구',
    '350': '노원구', '380': '은평구', '410': '서대문구', '440': '마포구', '470': '양천구',
    '500': '강서구', '530': '구로구', '545': '금천구', '560': '영등포구', '590': '동작구',
    '620': '관악구', '650': '서초구', '680': '강남구', '710': '송파구', '740': '강동구'
}
df["구"] = df["시군구코드"].map(gu_map)

# 동 매핑
if "법정동명" in df.columns:
    df["동"] = df["법정동명"]
else:
    df["동"] = np.nan

# ===== 3) 서울특별시만 필터링 =====
df_seoul = df[df["시"] == "서울특별시"].copy()


In [3]:
# ===== 4) 서울특별시 시·구·동 결측치 확인 =====
total_rows_seoul = len(df_seoul)
missing_cols_seoul = df_seoul[["시", "구", "동"]].isnull().sum()
missing_total_seoul = missing_cols_seoul.sum()

print("\n🏙️ [결측치 확인 - 서울특별시]")
print(f"📊 총 행 수: {total_rows_seoul:,}행")
print("🚫 결측치 수 (시·구·동 컬럼별):")
print(missing_cols_seoul)
print(f"🧾 전체 결측치 총합: {missing_total_seoul}개")

# ===== 5) 서울특별시 데이터 샘플 출력 =====
print("\n📌 [서울특별시 데이터 샘플 10건]")
print(df_seoul.head(10))


🏙️ [결측치 확인 - 서울특별시]
📊 총 행 수: 53행
🚫 결측치 수 (시·구·동 컬럼별):
시    0
구    0
동    0
dtype: int64
🧾 전체 결측치 총합: 0개

📌 [서울특별시 데이터 샘플 10건]
   연번  시도청    경찰서     치안센터명                   주소                 입력주소       X  \
0   1  서울청   서울중부    충4치안센터       서울 중구 충무로2길 39       서울 중구 충무로2길 39  199520   
1   2  서울청   서울종로    구기치안센터        서울 종로구 비봉길 13        서울 종로구 비봉길 13  196084   
2   3  서울청  서울서대문   북아현치안센터  서울 서대문구 북아현로 59-1번지  서울 서대문구 북아현로 59-1번지  195987   
3   4  서울청   서울혜화  종로3가치안센터       서울 종로구 돈화문로 28       서울 종로구 돈화문로 28  199254   
4   5  서울청   서울용산    청파치안센터     서울 용산구 청파로 274-2     서울 용산구 청파로 274-2  197359   
5   6  서울청   서울용산    용산치안센터        서울 용산구 신흥로 90        서울 용산구 신흥로 90  198645   
6   7  서울청   서울용산   한남2치안센터      서울 용산구 이태원로 281      서울 용산구 이태원로 281  200065   
7   8  서울청   서울용산    이촌치안센터     서울 용산구 이촌로71길 24     서울 용산구 이촌로71길 24  197570   
8   9  서울청   서울성북    삼선치안센터     서울 성북구 보문로29길 92     서울 성북구 보문로29길 92  201121   
9  10  서울청   서울성북   길음1치안센터   서울 성북구 정릉로 31길 30    서울 성북구 정릉로 3

In [4]:
# ===== 6) 최종 테이블 생성 (서울시 25개 자치구 포함) =====

# 6-1. 구별 설치 수 집계
df_gu_count = (
    df_seoul.groupby('구')
    .size()
    .reset_index(name='지역구별 치안센터 수')
    .sort_values(by='지역구별 치안센터 수', ascending=False)
)

# 6-2. 동별 설치 수 집계
df_dong_count = (
    df_seoul.groupby(['구', '동'])
    .size()
    .reset_index(name='동별 비상벨 설치수')
)

# 6-3. 컬럼 정리
df_result = df_dong_count.rename(columns={'구': '지역구별', '동': '동별'})
df_result = df_result[['지역구별', '동별', '동별 비상벨 설치수']]
df_result = df_result.sort_values(
    by=['지역구별', '동별 비상벨 설치수'],
    ascending=[True, False]
).reset_index(drop=True)

# 6-4. 서울 25개 자치구 기준으로 구별 값 보완
seoul_districts = [
    "종로구","중구","용산구","성동구","광진구","동대문구","중랑구","성북구","강북구","도봉구",
    "노원구","은평구","서대문구","마포구","양천구","강서구","구로구","금천구","영등포구",
    "동작구","관악구","서초구","강남구","송파구","강동구"
]
df_all_gu = pd.DataFrame({"구": seoul_districts})
df_all_gu = df_all_gu.merge(df_gu_count, on="구", how="left")
df_all_gu['지역구별 치안센터 수'] = df_all_gu['지역구별 치안센터 수'].fillna(0).astype(int)

# 6-5. 텍스트 요약
gu_summary_text = ", ".join(
    [f"{row['구']} {row['지역구별 치안센터 수']}개" for _, row in df_all_gu.iterrows()]
)

# 6-6. 출력
print("\n✅ [6단계] 최종 테이블 생성 완료")
print(f"📊 지역구별 총 설치수 요약: {gu_summary_text}")
print("\n📌 동별 수치화 데이터 샘플:")
print(df_result.head(20))


✅ [6단계] 최종 테이블 생성 완료
📊 지역구별 총 설치수 요약: 종로구 2개, 중구 1개, 용산구 4개, 성동구 1개, 광진구 2개, 동대문구 1개, 중랑구 1개, 성북구 4개, 강북구 3개, 도봉구 1개, 노원구 1개, 은평구 4개, 서대문구 1개, 마포구 3개, 양천구 1개, 강서구 2개, 구로구 5개, 금천구 2개, 영등포구 2개, 동작구 2개, 관악구 2개, 서초구 4개, 강남구 2개, 송파구 2개, 강동구 0개

📌 동별 수치화 데이터 샘플:
    지역구별    동별  동별 비상벨 설치수
0    강남구   대치동           1
1    강남구  압구정동           1
2    강북구   미아동           1
3    강북구   수유동           1
4    강북구   우이동           1
5    강서구   개화동           1
6    강서구   화곡동           1
7    관악구   봉천동           1
8    관악구   신림동           1
9    광진구   구의동           1
10   광진구   자양동           1
11   구로구   개봉동           2
12   구로구   구로동           2
13   구로구  신도림동           1
14   금천구   시흥동           2
15   노원구   하계동           1
16   도봉구   방학동           1
17  동대문구  답십리동           1
18   동작구   상도동           1
19   동작구   흑석동           1


In [5]:
# ===== 7) 결과 저장 =====
output_file_final = os.path.join(
    output_dir,
    "Y14_전처리_경찰청_서울특별시_치안센터_동별_비상벨설치수_20250630_TM.csv"
)
df_result.to_csv(output_file_final, index=False, encoding="utf-8-sig")

print(f"\n💾 [7단계] 결과 저장 완료")
print(f"📁 저장 경로: {output_file_final}")


💾 [7단계] 결과 저장 완료
📁 저장 경로: /Users/mac/Documents/SORA_Project/data/preprocessing/Y14_전처리_경찰청_서울특별시_치안센터_동별_비상벨설치수_20250630_TM.csv
